# Experiments with Progressive GANs

In [0]:
# Install the PyDrive wrapper & import libraries.
# This only needs to be done once per notebook.
!pip install -U -q PyDrive
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

# Authenticate and create the PyDrive client.
# This only needs to be done once per notebook.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

# Download a file based on its file ID.
#
# A file ID looks like: laggVyWshwcyP6kEI-y_W3P8D26sz
file_id = '1xce6ct41eKxTjyASRsKY8erSikFpVWG5'
X = drive.CreateFile({'id': file_id})
X = X.GetContentFile('model.pkl')

In [0]:
# Install the PyDrive wrapper & import libraries.
# This only needs to be done once per notebook.
!pip install -U -q PyDrive
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

# Authenticate and create the PyDrive client.
# This only needs to be done once per notebook.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

# Download a file based on its file ID.
#
# A file ID looks like: laggVyWshwcyP6kEI-y_W3P8D26sz
file_id = '188K19ucknC6wg1R6jbuPEhTq9zoufOx4'
X = drive.CreateFile({'id': file_id})
X = X.GetContentFile('model-celeb.pkl')

In [10]:
!git clone https://github.com/tkarras/progressive_growing_of_gans.git

Cloning into 'progressive_growing_of_gans'...
remote: Enumerating objects: 155, done.
remote: Total 155 (delta 0), reused 0 (delta 0), pack-reused 155
Receiving objects: 100% (155/155), 350.22 KiB | 1.43 MiB/s, done.
Resolving deltas: 100% (79/79), done.


In [0]:
!cp progressive_growing_of_gans/tfutil.py .

In [0]:
# Imports
import pickle
import numpy as np
import tensorflow as tf
import PIL.Image



In [0]:
tf.InteractiveSession()

In [13]:
with open('model-celeb.pkl', 'rb') as f:
    G, D, Gs = pickle.load(f)

AttributeError: ignored